### Get music library data
This script gets a list of all the music files in a directory path, along with basic metadata 
The musicbrainz API is then used to get more complete genre data for each artists 

In [1]:
import pandas as pd
import numpy as np
import musicbrainzngs as mbz 
import os
from tinytag import TinyTag
import csv

In [2]:
mbz.set_useragent('ArtistNetworkGraph', '0.1')

In [3]:
df0 = pd.DataFrame()

In [4]:
# Create empty lists to store results
song_list = []
album_list = []
artist_list = []
genre_list = []
year_list = []

seperator = '\\'
path = "C:\\Users\\User\\Dropbox\\00-MUSIC"

for root, dirs, files in os.walk(path):

    # select file name
    for file in files:
        # only get music files
        if file.endswith('.mp3' or '.aiff' or '.flac' or '.aac' or '.wav' or '.mp4' or '.m4a'): 
            
            # get genre
            tag: TinyTag = TinyTag.get(os.path.join(root, file))
            
            song_list.append(tag.title)
            artist_list.append(tag.artist)
            album_list.append(tag.album)
            genre_list.append(tag.genre)
             

In [5]:
df0['Song'] = song_list     
df0['Album'] = album_list     
df0['Artist'] = artist_list  
df0['Genre'] = genre_list 

In [6]:
#df0.to_csv('AllSongsData.csv')

In [7]:
artist_unique_1 = df0['Artist'].unique().tolist()
artist_unique_2 = [item for item in artist_unique_1 if item is not None]

In [8]:
artist_unique = [s.replace("\\", " ") for s in artist_unique_2]

In [9]:
df_a = pd.DataFrame()
df_a['Artist'] = artist_unique
df_a['Genre'] = ""
df_a['tag-list'] = ""
df_a['Artist2'] = ""

In [10]:
for i in range(0,len(df_a['Artist'])):
    #print(i)
    artist = df_a['Artist'][i]
    artist_get = mbz.search_artists(query=artist)['artist-list']       
    if bool(artist_get):
        
        artist_info = artist_get[0] 
        df_a['Artist2'][i] = artist_info['sort-name'] # to remove special chars
        if 'tag-list' in artist_info:
            df_a['tag-list'][i] = artist_info['tag-list']
        else:
            print('ERROR: ' + str(i) + ' TAGS not found')
                
    else:
        print('ERROR: ' + str(i) + ' not found in musicbrainz')

ERROR: 4 not found in musicbrainz
ERROR: 12 TAGS not found
ERROR: 13 TAGS not found
ERROR: 36 TAGS not found
ERROR: 37 TAGS not found
ERROR: 56 not found in musicbrainz
ERROR: 59 TAGS not found
ERROR: 62 TAGS not found
ERROR: 72 TAGS not found
ERROR: 73 TAGS not found
ERROR: 74 TAGS not found
ERROR: 75 TAGS not found
ERROR: 76 TAGS not found
ERROR: 77 TAGS not found
ERROR: 78 TAGS not found
ERROR: 80 TAGS not found
ERROR: 81 TAGS not found
ERROR: 85 TAGS not found
ERROR: 88 TAGS not found
ERROR: 91 TAGS not found
ERROR: 92 TAGS not found
ERROR: 94 TAGS not found
ERROR: 95 TAGS not found
ERROR: 97 TAGS not found
ERROR: 98 TAGS not found
ERROR: 101 TAGS not found
ERROR: 102 TAGS not found
ERROR: 112 TAGS not found
ERROR: 114 TAGS not found
ERROR: 137 TAGS not found
ERROR: 138 not found in musicbrainz
ERROR: 146 TAGS not found
ERROR: 148 TAGS not found
ERROR: 159 TAGS not found
ERROR: 171 TAGS not found
ERROR: 174 TAGS not found
ERROR: 187 TAGS not found
ERROR: 192 TAGS not found
ERROR: 1

In [11]:
# drop rows that don't have genre data
df_a1 = df_a[df_a['tag-list'] != '']

In [12]:
df_a2 = df_a1.reset_index(inplace=False, drop=True)

In [13]:
df_a2.Artist2.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [14]:
for i in range(0, len(df_a2)):
    #print(i)
    k = df_a2['tag-list'][i]
    
    # sort by genre tag count
    temp_df = pd.DataFrame(index=range(len(k)))
    temp_df['Count'] = ''
    temp_df['Name'] = ''

    for t in range(len(k)):   
        temp_df['Count'][t] = k[t]['count']
        temp_df['Name'][t] = k[t]['name']

    temp_sort = temp_df.sort_values(by='Count', ascending=False)    

    # create a list from the sorted genre tags
    temp_list = temp_sort['Name'].tolist()  
    df_a2['Genre'][i] = temp_list



In [15]:
# add song count for node size (represents proportion of music library)
df_a2['SongCount'] = ''
for i in range(0, len(df_a2['Artist'])):
    a = df_a2['Artist'][i]
    count = df0['Artist'].value_counts()[a]
    df_a2['SongCount'][i] = count
    

In [21]:
df2 = df_a2.dropna(subset=['Genre'])
df3 = df2.reset_index(drop='true')

In [22]:
df3['Artist'].duplicated().any()

False

In [23]:
df3['Artist2'].duplicated().any()  

### need to merge different instances of artists name

True

In [16]:
df_a2.to_csv('ArtistGenres.csv')

Output is the ArtistGenres.csv file passed to step 2 (Network Graph 2 notebook)

In [20]:
df_a2['Artist'].duplicated().any()

False